In [1]:
# Project Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Dict

# Paths
data_dir = '../data/jp_morgan'

In [2]:
# Load data
# aml_data = pd.read_csv(f'{data_dir}/aml_syn_data.csv')
fraud_data = pd.read_csv(f'{data_dir}/fraud_payment_data.csv')



In [3]:
fraud_data['Old_Txn_Type'] = fraud_data['Transaction_Type']
fraud_data['Transaction_Type'] = fraud_data['Transaction_Id'].str.split('-').str[:-1].str.join('-')
fraud_data['Transaction_Type'] = fraud_data['Old_Txn_Type'] + '-' + fraud_data['Transaction_Type']


In [4]:
fraud_data['Hybrid_Txn_Type'] = fraud_data['Old_Txn_Type'] + '-' + fraud_data['Transaction_Type']

In [ ]:
fraud_data['Hybrid_Txn_Type'].drop_duplicates()

In [6]:
common_columns = [
    'Time_step',
    'Label',
    'Transaction_Id',
    'Transaction_Type',
    'Sender_Id',
    'Sender_Account',
    'Sender_Country',
    'Bene_Id', 
    'Bene_Account',
    'Bene_Country',
    'USD_amount',
]
# aml_data = aml_data[common_columns]
fraud_data = fraud_data[common_columns]
aml_data = fraud_data.copy()

In [7]:
import polars as pl
df = pl.DataFrame(aml_data)

In [8]:
from ds_mapping import fraud
txn_mapping = fraud.txn_mapping

In [9]:
def std_map_transactions(df: pl.DataFrame, txn_mapping: dict) -> pl.DataFrame:
    # Pre-compute the mappings
    type_mapping = {k: v['std_txn_type'] for k, v in txn_mapping.items()}
    method_mapping = {k: v['std_txn_method'] for k, v in txn_mapping.items()}
    
    # First check for unmapped values
    unique_txn_types = df.get_column('Transaction_Type').unique().to_list()  # Note the column name change
    unmapped_txns = [txn for txn in unique_txn_types if txn not in type_mapping]
    
    if unmapped_txns:
        raise ValueError(
            f"Found {len(unmapped_txns)} unmapped transaction types:\n"
            f"{unmapped_txns}\n"
            f"Please update the mapping dictionary with these values."
        )
    
    # If we get here, all values are mapped, so proceed with mapping
    return df.with_columns([
        pl.col('Transaction_Type').replace(type_mapping).alias('std_txn_type'),
        pl.col('Transaction_Type').replace(method_mapping).alias('std_txn_method')
    ])

df = std_map_transactions(df, txn_mapping)

Feature Columns:
* hour of transaction
* normalised age on transaction

Data Preprocessing
1. Filter only transactions considering money movements
2. Retable DataFrame to be in Party v Counterparty format

In [10]:
# AML Data Preprocessing
# Filter only money movements
# aml_data = aml_data[aml_data['Transaction_Type'].isin(['WIRE', 'PAYMENT', 'WITHDRAWL', 'CASH-DEPOSIT', 'CRYPTO-TRANSFER'])].reset_index()

# Identify if row is reversed for party v counterparty retabling
def pd_party_reverse_flag(
        df: pd.DataFrame, 
        sender_col: str = 'Sender_Id', 
        bene_col: str = 'Bene_Id', 
        transaction_col: str = 'Transaction_Type'
        ) -> pd.DataFrame:

    df.loc[:, 'sender_type'] = df[sender_col].str.split('-').str[:2].str.join('-')
    df.loc[:, 'benef_type'] = df[bene_col].str.split('-').str[:2].str.join('-')
    condition = (
        (df[transaction_col] == 'CASH-DEPOSIT') |
        (((df[transaction_col] == 'MAKE-PAYMENT') | (df[transaction_col] == 'QUICK-PAYMENT') | (df[transaction_col] == 'MOVE-FUNDS')) & (df['benef_type'] == 'JPMC-CLIENT'))
    )
    df.loc[:, 'is_credit'] = condition
    df.loc[:, 'reversed'] = condition
    return df

def pl_party_reverse_flag(
        df: pl.DataFrame, 
        sender_col: str = 'Sender_Id', 
        bene_col: str = 'Bene_Id', 
        transaction_col: str = 'Transaction_Type'
        ) -> pl.DataFrame:

    df = df.with_columns([
        pl.col(sender_col).str.split('-').list.slice(0, 2).list.join('-').alias('sender_type'),
        pl.col(bene_col).str.split('-').list.slice(0, 2).list.join('-').alias('benef_type')
    ])
    
    condition = (
        (pl.col(transaction_col) == 'DEPOSIT') |
        (((pl.col(transaction_col) == 'PAYMENT') | 
          (pl.col(transaction_col) == 'TRANSFER')) & 
         (pl.col('benef_type') == 'JPMC-CLIENT'))
    )
    
    df = df.with_columns([
        condition.alias('is_credit'),
        condition.alias('reversed')
    ])
    
    return df

df = pl_party_reverse_flag(df, transaction_col='std_txn_type')

In [11]:
# Create party v counterparty retabling
def pd_new_table(
        df: pd.DataFrame, 
        rev_col: str = 'reversed', 
        retained_cols: Optional[List[str]] = None
        ) -> pd.DataFrame:

    for type in ('Id', 'Account', 'Country'):
        df.loc[:, f'party_{type}'] = df[f'Sender_{type}'].where(~df[rev_col], df[f'Bene_{type}'])
        df.loc[:, f'cparty_{type}'] = df[f'Bene_{type}'].where(~df[rev_col], df[f'Sender_{type}'])
    retained_cols.extend(['party_Id', 'party_Account', 'party_Country',
                          'cparty_Id', 'cparty_Account', 'cparty_Country'])
    new_table = df[retained_cols]
    return new_table

def pl_new_table(
        df: pl.DataFrame, 
        rev_col: str = 'reversed', 
        retained_cols: Optional[List[str]] = None
        ) -> pl.DataFrame:

    if retained_cols is None:
        retained_cols = []

    for type in ('Id', 'Account', 'Country'):
        df = df.with_columns([
            pl.when(~pl.col(rev_col)).then(pl.col(f'Sender_{type}')).otherwise(pl.col(f'Bene_{type}')).alias(f'party_{type}'),
            pl.when(~pl.col(rev_col)).then(pl.col(f'Bene_{type}')).otherwise(pl.col(f'Sender_{type}')).alias(f'cparty_{type}')
        ])
    
    retained_cols.extend(['party_Id', 'party_Account', 'party_Country',
                          'cparty_Id', 'cparty_Account', 'cparty_Country'])
    
    new_table = df.select(retained_cols)
    return new_table


df = pl_new_table(df, retained_cols=[
    'Time_step', 
    'Label',
    'Transaction_Id',
    'Transaction_Type',
    'std_txn_type',
    'std_txn_method',
    'is_credit',
    'USD_amount'
    ])

3. Normalise dates (change into age of account on transaction) and extract hour/minute for off hours transaction

In [12]:
# Normalise date and extract hour minutes
def pd_feature_time(
        df: pd.DataFrame, 
        dt_column: str = 'Time_step'
        ) -> pd.DataFrame:
    
    time_series = pd.to_datetime(df[dt_column])
    df['txn_time_hr'] = time_series.dt.hour
    df['txn_time_mm'] = time_series.dt.minute
    
    return df

def pd_normalise_date(df, dt_column='Time_step', primary_col='party_Id'):
    time_series = pd.to_datetime(df[dt_column]).dt.date
    df['txn_age_days'] = (time_series - time_series.groupby(df[primary_col]).transform('min')).astype('timedelta64[ns]').dt.days
    
    return df

def pl_feature_time(
        df: pl.DataFrame, 
        dt_column: str = 'Time_step'
        ) -> pl.DataFrame:
    
    df = df.with_columns([
        pl.col(dt_column).str.strptime(pl.Datetime).alias('parsed_time')
    ])
    
    df = df.with_columns([
        pl.col('parsed_time').dt.hour().alias('txn_time_hr'),
        pl.col('parsed_time').dt.minute().alias('txn_time_mm')
    ])
    
    return df.drop('parsed_time')

def pl_normalise_date(
        df: pl.DataFrame, 
        dt_column: str = 'Time_step', 
        primary_col: str = 'party_Id',
        dt_fmt: str = '%Y-%m-%d %H:%M:%S'
        ) -> pl.DataFrame:
    
    df = df.with_columns([
        pl.col(dt_column).str.strptime(pl.Date, format=dt_fmt).alias('parsed_date')
    ])
    
    min_dates = df.group_by(primary_col).agg(pl.col('parsed_date').min().alias('min_date'))
    df = df.join(min_dates, on=primary_col)
    
    df = df.with_columns([
        (pl.col('parsed_date') - pl.col('min_date')).dt.total_days().alias('txn_age_days')
    ])
    
    return df.drop(['parsed_date', 'min_date'])

df = pl_feature_time(df)
df = pl_normalise_date(df)


4. Collect previous transaction details: (transaction type, amount, if applicable delta between transactions)

In [13]:
# Extract previous transaction details
dt_norm_values = {
    'day': 'txn_age_days',
    'hour': 'txn_time_hr',
    'min': 'txn_time_mm',
}
def pd_fill_prev_txn(
        df: pd.DataFrame, 
        primary_key: str = 'party_Id', 
        dt_column: str | None = None,
        dt_norm_columns: Dict[str, str] | None = None, 
        columns_to_fill: List[str] = [], 
        shift: int = 1, 
        include_age_delta: bool = False
        ) -> pd.DataFrame:
    
    if dt_column != None:
        core_columns = [primary_key, dt_column]
        temp_df = df[core_columns + columns_to_fill].copy()

        temp_df[dt_column] = pd.to_datetime(temp_df[dt_column])
        temp_df = temp_df.sort_values([primary_key, dt_column])
        
        temp_df['prev_age_delta'] = (
            (temp_df[dt_column] - temp_df.groupby(primary_key)[dt_column].shift(shift))
            .dt.total_seconds())/(24 * 3600)

        
    else:
        dt_norm_cols = [dt_norm_columns['day'], dt_norm_columns['hour'], dt_norm_columns['min']]
        core_columns = [primary_key] + dt_norm_cols
        temp_df = df[core_columns + columns_to_fill].copy()
        
        temp_df = temp_df.sort_values(by=[primary_key] + dt_norm_cols, ascending=True)
        temp_df['rel_age'] = (
            temp_df[dt_norm_columns['day']] + 
            (temp_df[dt_norm_columns['hour']] / 24) + 
            (temp_df[dt_norm_columns['min']] / (24 * 60))
            )
        temp_df['prev_age_delta'] = temp_df['rel_age'] - temp_df.groupby(primary_key)['rel_age'].shift(shift)
    
    new_columns = ['prev_' + col for col in columns_to_fill]

    temp_df[new_columns] = temp_df.groupby(primary_key)[columns_to_fill].shift(shift)
    if include_age_delta:
        new_columns.append('prev_age_delta')
    
    return df.merge(temp_df[new_columns], left_index=True, right_index=True)

def pl_fill_prev_txn(
        df: pl.DataFrame, 
        primary_key: str = 'party_Id', 
        dt_column: str | None = None,
        dt_norm_columns: Dict[str, str] | None = None, 
        columns_to_fill: List[str] = [], 
        shift: int = 1, 
        include_age_delta: bool = False
        ) -> pl.DataFrame:
    
    if dt_column is not None:
        core_columns = [primary_key, dt_column]
        temp_df = df.select(core_columns  + columns_to_fill)

        temp_df = temp_df.with_columns([
            pl.col(dt_column).str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S").alias(dt_column)
        ])
        temp_df = temp_df.sort([primary_key, dt_column])
        
        temp_df = temp_df.with_columns([
            ((pl.col(dt_column) - pl.col(dt_column).shift(shift)).dt.seconds() / (24 * 3600)).alias('prev_age_delta')
        ])
        
    else:
        dt_norm_cols = [dt_norm_columns['day'], dt_norm_columns['hour'], dt_norm_columns['min']]
        core_columns = [primary_key] + dt_norm_cols
        temp_df = df.select(core_columns + columns_to_fill)
        
        temp_df = temp_df.sort([primary_key] + dt_norm_cols)
        temp_df = temp_df.with_columns([
            (pl.col(dt_norm_columns['day']) + 
             (pl.col(dt_norm_columns['hour']) / 24) + 
             (pl.col(dt_norm_columns['min']) / (24 * 60))).alias('rel_age')
        ])
        temp_df = temp_df.with_columns([
            (pl.col('rel_age') - pl.col('rel_age').shift(shift)).alias('prev_age_delta')
        ])
    
    new_columns = ['prev_' + col for col in columns_to_fill]

    temp_df = temp_df.with_columns([
        pl.col(col).shift(shift).alias('prev_' + col) for col in columns_to_fill
    ])
    
    if include_age_delta:
        new_columns.append('prev_age_delta')
    
    return df.join(temp_df.select([primary_key] + new_columns), on=primary_key, how='left')

pd_df = df.to_pandas()
pd_df = pd_fill_prev_txn(pd_df, dt_norm_columns=dt_norm_values, columns_to_fill=['std_txn_type', 'std_txn_method', 'USD_amount'], include_age_delta=True)
# df = pl_fill_prev_txn(df, dt_norm_columns=dt_norm_values, columns_to_fill=['std_txn_type', 'USD_amount'], include_age_delta=True)


In [ ]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
from tqdm.auto import tqdm
import multiprocessing as mp
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

class TransactionProcessor:
    def __init__(
        self,
        time_bins: List[int] = [7, 14, 30],
        amount_col: str = 'USD_amount',  # Updated
        customer_id_col: str = 'party_Id',  # Updated
        age_col: str = 'txn_age_days',  # Updated
        reporting_threshold: int = 10000 # Added
    ):
        self.time_bins = time_bins
        self.amount_col = amount_col
        self.customer_id_col = customer_id_col
        self.age_col = age_col
        self.reporting_threshold = reporting_threshold

    @staticmethod
    def calculate_customer_metrics(args: Tuple[pd.DataFrame, Dict]) -> pd.DataFrame:
        """Calculate metrics for a single customer's transactions"""
        customer_data, config = args
        result = customer_data.copy()
        
        # Get configuration
        time_bins = config['time_bins']
        amount_col = config['amount_col']
        age_col = config['age_col']
        threshold = config.get('reporting_threshold', 10000)  # Add to config

        
        # Sort by age
        result = result.sort_values(age_col)

        # Calculate suspicious range (80-99.9% of threshold)
        suspicious_lower = threshold * 0.8
        suspicious_upper = threshold

        
        # Calculate metrics for each time bin
        for days in time_bins:
            result[f'volume_{days}d_sum'] = 0.0
            result[f'velocity_{days}d_count'] = 0
            result[f'stat_{days}d_median'] = 0.0
            result[f'stat_{days}d_mad'] = 0.0        
            result[f'under_threshold_{days}d_count'] = 0  # New metric
            result[f'under_threshold_{days}d_sum'] = 0.0  # New metric

            for idx in range(len(result)):
                current_age = result.iloc[idx][age_col]
                mask = (
                    (result[age_col] <= (current_age - 1)) & 
                    (result[age_col] > (current_age - days - 1))
                )
                window_data = result.loc[mask, amount_col]
                suspicious_data = window_data[
                    (window_data >= suspicious_lower) & 
                    (window_data < suspicious_upper)
                ]

                if len(window_data) > 0:
                    result.iloc[idx, result.columns.get_loc(f'volume_{days}d_sum')] = window_data.sum()
                    median_ = window_data.median()
                    mad_ = np.median([abs(x - median_) for x in window_data])
                    result.iloc[idx, result.columns.get_loc(f'stat_{days}d_median')] = median_
                    result.iloc[idx, result.columns.get_loc(f'stat_{days}d_mad')] = mad_
                    result.iloc[idx, result.columns.get_loc(f'velocity_{days}d_count')] = len(window_data)
                    result.iloc[idx, result.columns.get_loc(f'under_threshold_{days}d_count')] = len(suspicious_data)
                    result.iloc[idx, result.columns.get_loc(f'under_threshold_{days}d_sum')] = suspicious_data.sum()

        
        return result

    def process_transactions(
        self,
        df: pd.DataFrame,
        n_jobs: int = None,
        batch_size: int = 1000
    ) -> pd.DataFrame:
        """Process all transactions using parallel processing"""
        if n_jobs is None:
            n_jobs = max(1, mp.cpu_count() - 1)
        
        print(f"Starting processing with {n_jobs} workers")
        print(f"Total transactions: {len(df):,}")
        print(f"Total customers: {df[self.customer_id_col].nunique():,}")
        
        # Sort data
        df = df.sort_values([self.customer_id_col, self.age_col])
        
        # Split customers into batches
        all_customers = df[self.customer_id_col].unique()
        customer_batches = [
            all_customers[i:i + batch_size] 
            for i in range(0, len(all_customers), batch_size)
        ]
        
        results = []
        config = {
            'time_bins': self.time_bins,
            'amount_col': self.amount_col,
            'age_col': self.age_col,
            'reporting_threshold': self.reporting_threshold
        }
        
        print(f"Processing {len(customer_batches)} batches...")
        
        for batch_idx, customer_batch in enumerate(customer_batches, 1):
            print(f"\nProcessing batch {batch_idx}/{len(customer_batches)}")
            
            # Get data for current batch
            batch_data = df[df[self.customer_id_col].isin(customer_batch)]
            
            # Process this batch without multiprocessing
            batch_results = []
            for _, group in tqdm(batch_data.groupby(self.customer_id_col), 
                               desc="Processing customers in batch"):
                result = self.calculate_customer_metrics((group, config))
                batch_results.append(result)
            
            # Combine batch results
            if batch_results:
                batch_df = pd.concat(batch_results, ignore_index=True)
                results.append(batch_df)
            
            # Save intermediate results
            if batch_idx % 5 == 0:  # Save every 5 batches
                interim_df = pd.concat(results, ignore_index=True)
                interim_df.to_parquet(f'interim_results_batch_{batch_idx}.parquet')
            
            # Clear memory
            del batch_data, batch_results
            gc.collect()
        
        # Combine all results
        print("\nCombining all results...")
        result_df = pd.concat(results, ignore_index=True)
        
        # Calculate ratios
        # print("Calculating ratios...")
        # for days1, days2 in [(7, 14), (7, 30), (14, 30)]:
        #     # Average ratio
        #     result_df[f'volume_avg_{days1}d_to_{days2}d_ratio'] = (
        #         result_df[f'volume_{days1}d_avg'] / 
        #         result_df[f'volume_{days2}d_avg'].replace(0, np.nan)
        #     ).fillna(0)
            
        #     # Count ratio
        #     result_df[f'volume_count_{days1}d_to_{days2}d_ratio'] = (
        #         (result_df[f'volume_{days1}d_count'] / days1) /
        #         (result_df[f'volume_{days2}d_count'] / days2).replace(0, np.nan)
        #     ).fillna(0)
        
        return result_df

def process_with_monitoring(
    df: pd.DataFrame,
    time_bins: List[int] = [7, 14, 30],
    batch_size: int = 1000
) -> pd.DataFrame:
    """Process with monitoring"""
    import psutil
    import os
    
    process = psutil.Process(os.getpid())
    start_time = datetime.now()
    
    print(f"Start time: {start_time}")
    print(f"Initial memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")
    
    try:
        # Initialize processor
        processor = TransactionProcessor(
            time_bins=time_bins,
            amount_col='USD_amount',
            customer_id_col='party_Id',
            age_col='txn_age_days',
            reporting_threshold=10000
        )
        
        # Process data
        result_df = processor.process_transactions(
            df=df,
            batch_size=batch_size
        )
        
        end_time = datetime.now()
        processing_time = end_time - start_time
        
        print("\nProcessing completed successfully!")
        print(f"End time: {end_time}")
        print(f"Total processing time: {processing_time}")
        print(f"Final memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")
        
        return result_df
        
    except Exception as e:
        print(f"\nError during processing: {str(e)}")
        raise

# Example usage
if __name__ == "__main__":
    # Read your data
    df = pd_df.copy()  # or however you load your data
    
    # Process with monitoring
    result_df = process_with_monitoring(
        df=df,
        time_bins=[7, 14, 30],
        batch_size=1000  # Process 1000 customers at a time
    )
    
    # Save final results
    result_df.to_parquet('../data/jp_morgan/transaction_metrics_final_fraud_v3.parquet')

In [15]:
calculator = TransactionStatsCalculator(
        time_bins=[7, 14, 30],
        metrics=['avg', 'std', 'count'],
        ratio_pairs=[(7, 14), (7, 30), (14, 30)]
    )


In [ ]:
test_data = calculator.process_transactions(
    df=aml_data,
    amount_col='USD_amount',
    customer_id_col='party_Id',
    age_col='txn_age_days'
)

In [ ]:
test_data.to_parquet('../data/test.parquet')

In [ ]:
aml_data['USD_amount'].max()

In [10]:
test_b = aml_data[:10].copy()
test_b.columns = ['b_' + x for x in test_b.columns]
test_a = aml_data[:10].copy()

In [9]:
df = aml_data.copy()

In [19]:
df = feature_time(df, 'Time_step')

In [11]:
df['Time_step']
timestamp = pd.to_datetime(df['Time_step'])

In [ ]:
aml_senders = aml_data[['Sender_Id', 'Sender_Account']].drop_duplicates().
fraud_senders = fraud_data[['Sender_Id', 'Sender_Account']].drop_duplicates()

In [ ]:
def plot_hist_df(df):
    df['date'] = pd.to_datetime(df['Time_step'])
    # Extract the year and month
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month

    # Plot the distribution of months per year
    df.groupby(['year', 'month']).size().unstack().plot(kind='bar', stacked=True)

    plt.xlabel('Year')
    plt.ylabel('Count of Entries')
    plt.title('Month Distribution per Year')
    plt.show()

plot_hist_df(aml_data)

In [ ]:
aml_data['Label'].map({'GOOD': 0, 'BAD': 1})

In [ ]:
def process_with_checkpoints(df):
    # Create checkpoint directory
    import os
    os.makedirs('checkpoints', exist_ok=True)
    
    try:
        result_df = process_with_monitoring(
            df=df,
            customer_id_col='party_Id',
            amount_col='USD_amount',
            age_col='txn_age_days',
            n_jobs=mp.cpu_count() - 1  # Use all but one CPU
        )
        
        # Save final result
        result_df.to_parquet('checkpoints/final_result.parquet')
        print("Results saved to 'checkpoints/final_result.parquet'")
        
        return result_df
        
    except KeyboardInterrupt:
        print("\nProcessing interrupted by user")
        return None
    
process_with_checkpoints(aml_data)